In [654]:
import numpy_financial as npf
import PortfolioLab as pl
import pandas as pd
import numpy as np

from dateutil.relativedelta import relativedelta

In [655]:
%%html
<style>
.dataframe td {
    white-space: nowrap;
}
</style>

In [656]:
npf.rate(nper=23, pmt=-40000, pv=-510000, fv=2000000, when=1)

0.02057021890678331

In [657]:
Periods = 23
Cashflow_constant = 40000
Initial_portfolio = 510000
Objective = 2000000
inflation = 0.02
tax = 0.28
begin = '2019-06-30'
end= '2041-06-30'

rate_nt_ni = npf.rate(nper=Periods, pmt=Cashflow_constant * -1, pv=Initial_portfolio * -1, fv=Objective, when=1)
rate_nt_ni

0.02057021890678331

In [658]:
rate_nt_i = ((1 + rate_nt_ni) * (1 + inflation)) - 1
rate_nt_i

0.040981623284918944

In [659]:
rate_t_i = rate_nt_i / (1 - tax)
rate_t_i

0.05691892122905409

In [707]:
%%time

pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = '{:,.3f}'.format

Periods = 23
Cashflow_constant = 40000
Initial_portfolio = 510000
Objective = 2000000
inflation = 0.02
tax = 0.28
begin = '2019-03-30'

rate_nt_ni = npf.rate(nper=Periods, pmt=Cashflow_constant * -1, pv=Initial_portfolio * -1, fv=Objective, when=1)

rate_nt_i = ((1 + rate_nt_ni) * (1 + inflation)) - 1

rate_t_i = rate_nt_i / (1 - tax)





begin_as_date = pd.to_datetime(begin)

dates_list=[]

for i in np.arange(1, Periods + 1):
    i-=1
    new_date = begin_as_date + relativedelta(years=+i)
    dates_list.append(new_date)

df = pd.DataFrame()

df['Período'] = np.arange(1, Periods + 1)
df['Data'] = dates_list
df['Valor Inicial'] = Initial_portfolio
df['Reforços (Início do período)'] = Cashflow_constant
df['Reforços (Cumulativo)'] = df['Reforços (Início do período)'].cumsum()
df['Valor Inicial + Reforços (início de período)'] = df['Valor Inicial'] + df['Reforços (Cumulativo)']
df['Valor carteira inicio do período'] = df['Valor Inicial']  + Cashflow_constant
df['Rent. Necessária'] = rate_t_i
df['Retorno Anual'] = df['Valor carteira inicio do período'] * df['Rent. Necessária']
df['Valor esperado final período'] = df['Valor carteira inicio do período'] + df['Retorno Anual']

for i in np.arange(1, Periods):
    df['Valor carteira inicio do período'].iloc[i] = df['Valor esperado final período'].iloc[i-1] + Cashflow_constant
    df['Retorno Anual'].iloc[i] = df['Valor carteira inicio do período'].iloc[i] * rate_t_i
    df['Valor esperado final período'].iloc[i] = df['Retorno Anual'].iloc[i] + df['Valor carteira inicio do período'].iloc[i]
    
df['Ganhos (Cumulativos)'] = df['Retorno Anual'].cumsum()
df['Imposto'] = tax
df['Impostos potenciais (Cumulativos)'] = df['Imposto'] * df['Ganhos (Cumulativos)']
df['Valor carteira final do período depois de impostos'] = df['Valor esperado final período'] - df['Impostos potenciais (Cumulativos)']
df['Valor carteira início do período depois de impostos'] = Initial_portfolio + Cashflow_constant

for i in np.arange(1, Periods):
    df['Valor carteira início do período depois de impostos'].iloc[i] = df['Valor carteira final do período depois de impostos'].iloc[i-1] + Cashflow_constant
    
df['Inflação esperada (por período)'] = 1 + inflation
df['Inflação esperada (por período)'].iloc[0] = 1
df['Deflator'] = df['Inflação esperada (por período)'].cumprod() * 100

df['Valor carteira NT NI'] = (df['Valor carteira final do período depois de impostos'] / df['Deflator']) * 100
    
df

Wall time: 22.9 ms


,Período,Data,Valor Inicial,Reforços (Início do período),Reforços (Cumulativo),Valor Inicial + Reforços (início de período),Valor carteira inicio do período,Rent. Necessária,Retorno Anual,Valor esperado final período,Ganhos (Cumulativos),Imposto,Impostos potenciais (Cumulativos),Valor carteira final do período depois de impostos,Valor carteira início do período depois de impostos,Inflação esperada (por período),Deflator,Valor carteira NT NI
0,1,2019-03-30,510000,40000,40000,550000,"550,000.000",0.057,"31,305.407","581,305.407","31,305.407",0.280,"8,765.514","572,539.893","550,000.000",1.000,100.000,"572,539.893"
1,2,2020-03-30,510000,40000,80000,590000,"621,305.407",0.057,"35,364.034","656,669.440","66,669.440",0.280,"18,667.443","638,001.997","612,539.893",1.020,102.000,"625,492.154"
2,3,2021-03-30,510000,40000,120000,630000,"696,669.440",0.057,"39,653.673","736,323.113","106,323.113",0.280,"29,770.472","706,552.641","678,001.997",1.020,104.040,"679,116.341"
3,4,2022-03-30,510000,40000,160000,670000,"776,323.113",0.057,"44,187.474","820,510.587","150,510.587",0.280,"42,142.964","778,367.623","746,552.641",1.020,106.121,"733,473.196"
4,5,2023-03-30,510000,40000,200000,710000,"860,510.587",0.057,"48,979.334","909,489.922","199,489.922",0.280,"55,857.178","853,632.744","818,367.623",1.020,108.243,"788,624.706"
5,6,2024-03-30,510000,40000,240000,750000,"949,489.922",0.057,"54,043.942","1,003,533.864","253,533.864",0.280,"70,989.482","932,544.382","893,632.744",1.020,110.408,"844,634.178"
6,7,2025-03-30,510000,40000,280000,790000,"1,043,533.864",0.057,"59,396.822","1,102,930.685","312,930.685",0.280,"87,620.592","1,015,310.094","972,544.382",1.020,112.616,"901,566.307"
7,8,2026-03-30,510000,40000,320000,830000,"1,142,930.685",0.057,"65,054.382","1,207,985.067","377,985.067",0.280,"105,835.819","1,102,149.248","1,055,310.094",1.020,114.869,"959,487.246"
8,9,2027-03-30,510000,40000,360000,870000,"1,247,985.067",0.057,"71,033.964","1,319,019.031","449,019.031",0.280,"125,725.329","1,193,293.702","1,142,149.248",1.020,117.166,"1,018,464.685"
9,10,2028-03-30,510000,40000,400000,910000,"1,359,019.031",0.057,"77,353.897","1,436,372.928","526,372.928",0.280,"147,384.420","1,288,988.508","1,233,293.702",1.020,119.509,"1,078,567.922"


In [708]:
from pandas.tseries.offsets import Day

In [709]:
%%time

df2 = df[['Data', 'Valor carteira início do período depois de impostos', 'Deflator']]
df2['Deflator'] = df2['Deflator'].shift()
df2['Deflator'].iloc[0] = 100
df2['Data'] = df2['Data'] + DateOffset(days=1)
df2['Data'] = np.where(df2['Data'].dt.day_name().isin(['Sunday']), df2['Data'] + Day(1), df2['Data'])
df2['Data'] = np.where(df2['Data'].dt.day_name().isin(['Saturday']), df2['Data'] + Day(2), df2['Data'])

df2.columns = ['Data', 'Valor carteira', 'Deflator']

df2

Wall time: 5.96 ms


,Data,Valor carteira,Deflator
0,2019-04-01,"550,000.000",100.000
1,2020-03-31,"612,539.893",100.000
2,2021-03-31,"678,001.997",102.000
3,2022-03-31,"746,552.641",104.040
4,2023-03-31,"818,367.623",106.121
5,2024-04-01,"893,632.744",108.243
6,2025-03-31,"972,544.382",110.408
7,2026-03-31,"1,055,310.094",112.616
8,2027-03-31,"1,142,149.248",114.869
9,2028-03-31,"1,233,293.702",117.166


In [710]:
%%time

from pandas.tseries.offsets import DateOffset

df3 = df[['Data', 'Valor carteira final do período depois de impostos', 'Deflator']]
df3['Deflator'] = df3['Deflator'].shift()
df3['Deflator'].iloc[0] = 100
df3['Data'] = df3['Data'] + DateOffset(months=12)
df3['Data'] = np.where(df3['Data'].dt.day_name().isin(['Sunday']), df3['Data'] - Day(2), df3['Data'])
df3['Data'] = np.where(df3['Data'].dt.day_name().isin(['Saturday']), df3['Data'] - Day(1), df3['Data'])

df3['Deflator'] = df3['Deflator'].shift(-1)
df3['Deflator'].iloc[-1] = df3['Deflator'].iloc[-2] * (1 + inflation)

df3.columns = ['Data', 'Valor carteira', 'Deflator']

df3

Wall time: 4.96 ms


,Data,Valor carteira,Deflator
0,2020-03-30,"572,539.893",100.000
1,2021-03-30,"638,001.997",102.000
2,2022-03-30,"706,552.641",104.040
3,2023-03-30,"778,367.623",106.121
4,2024-03-29,"853,632.744",108.243
5,2025-03-28,"932,544.382",110.408
6,2026-03-30,"1,015,310.094",112.616
7,2027-03-30,"1,102,149.248",114.869
8,2028-03-30,"1,193,293.702",117.166
9,2029-03-30,"1,288,988.508",119.509


In [711]:
df_chart = pd.concat([df2, df3]).sort_values(by='Data').reset_index().drop('index', axis=1)

df_chart['Valor carteira'] = (df_chart['Valor carteira'] / df_chart['Deflator']) * 100

df_chart.drop('Deflator', axis=1, inplace=True)
df_chart.set_index('Data', inplace=True)

pl.ichart(df_chart)

In [706]:
%%timeit

pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = '{:,.3f}'.format

from pandas.tseries.offsets import DateOffset

Periods = 23
Cashflow_constant = 40000
Initial_portfolio = 510000
Objective = 2000000
inflation = 0.02
tax = 0.28
begin = '2019-06-30'
end= '2041-06-30'

rate_nt_ni = npf.rate(nper=Periods, pmt=Cashflow_constant * -1, pv=Initial_portfolio * -1, fv=Objective, when=1)

rate_nt_i = ((1 + rate_nt_ni) * (1 + inflation)) - 1

rate_t_i = rate_nt_i / (1 - tax)





begin_as_date = pd.to_datetime(begin)

dates_list=[]

for i in np.arange(1, Periods + 1):
    i-=1
    new_date = begin_as_date + relativedelta(years=+i)
    dates_list.append(new_date)

df = pd.DataFrame()

df['Período'] = np.arange(1, Periods + 1)
df['Data'] = dates_list
df['Valor Inicial'] = Initial_portfolio
df['Reforços (Início do período)'] = Cashflow_constant
df['Reforços (Cumulativo)'] = df['Reforços (Início do período)'].cumsum()
df['Valor Inicial + Reforços (início de período)'] = df['Valor Inicial'] + df['Reforços (Cumulativo)']
df['Valor carteira inicio do período'] = df['Valor Inicial']  + Cashflow_constant
df['Rent. Necessária'] = rate_t_i
df['Retorno Anual'] = df['Valor carteira inicio do período'] * df['Rent. Necessária']
df['Valor esperado final período'] = df['Valor carteira inicio do período'] + df['Retorno Anual']

for i in np.arange(1, Periods):
    df['Valor carteira inicio do período'].iloc[i] = df['Valor esperado final período'].iloc[i-1] + Cashflow_constant
    df['Retorno Anual'].iloc[i] = df['Valor carteira inicio do período'].iloc[i] * rate_t_i
    df['Valor esperado final período'].iloc[i] = df['Retorno Anual'].iloc[i] + df['Valor carteira inicio do período'].iloc[i]
    
df['Ganhos (Cumulativos)'] = df['Retorno Anual'].cumsum()
df['Imposto'] = tax
df['Impostos potenciais (Cumulativos)'] = df['Imposto'] * df['Ganhos (Cumulativos)']
df['Valor carteira final do período depois de impostos'] = df['Valor esperado final período'] - df['Impostos potenciais (Cumulativos)']
df['Valor carteira início do período depois de impostos'] = Initial_portfolio + Cashflow_constant

for i in np.arange(1, Periods):
    df['Valor carteira início do período depois de impostos'].iloc[i] = df['Valor carteira final do período depois de impostos'].iloc[i-1] + Cashflow_constant
    
df['Inflação esperada (por período)'] = 1 + inflation
df['Inflação esperada (por período)'].iloc[0] = 1
df['Deflator'] = df['Inflação esperada (por período)'].cumprod() * 100

df['Valor carteira NT NI'] = (df['Valor carteira final do período depois de impostos'] / df['Deflator']) * 100
    

    
    
    
df2 = df[['Data', 'Valor carteira início do período depois de impostos', 'Deflator']]
df2['Deflator'] = df2['Deflator'].shift()
df2['Deflator'].iloc[0] = 100
df2['Data'] = df2['Data'] + DateOffset(days=1)
df2['Data'] = np.where(df2['Data'].dt.day_name().isin(['Sunday']), df2['Data'] + Day(1), df2['Data'])
df2['Data'] = np.where(df2['Data'].dt.day_name().isin(['Saturday']), df2['Data'] + Day(2), df2['Data'])

df2.columns = ['Data', 'Valor carteira', 'Deflator']


df3 = df[['Data', 'Valor carteira final do período depois de impostos', 'Deflator']]
df3['Deflator'] = df3['Deflator'].shift()
df3['Deflator'].iloc[0] = 100
df3['Data'] = df3['Data'] + DateOffset(months=12)
df3['Data'] = np.where(df3['Data'].dt.day_name().isin(['Sunday']), df3['Data'] - Day(2), df3['Data'])
df3['Data'] = np.where(df3['Data'].dt.day_name().isin(['Saturday']), df3['Data'] - Day(1), df3['Data'])

df3['Deflator'] = df3['Deflator'].shift(-1)
df3['Deflator'].iloc[-1] = df3['Deflator'].iloc[-2] * (1 + inflation)

df3.columns = ['Data', 'Valor carteira', 'Deflator']


df_chart = pd.concat([df2, df3]).sort_values(by='Data').reset_index().drop('index', axis=1)

df_chart['Valor carteira'] = (df_chart['Valor carteira'] / df_chart['Deflator']) * 100

df_chart.drop('Deflator', axis=1, inplace=True)
df_chart.set_index('Data', inplace=True)

28.7 ms ± 375 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
